# Risk Report Asset Allocation

In [1]:
import pandas as pd
from numpy import NaN

In [3]:
# Specify the date and portfolio here
date = '20200529'
portfolio = '19437'
mode='production'
reportingCurrency = 'USD'

## Load Portfolio

1. Load portfolio;
2. Add security id.

In [17]:
from risk_report.data import getIdnType, getPortfolioPositions
from utils.dframe import dictListToDataFrame

df = dictListToDataFrame(getPortfolioPositions(portfolio, date, mode))
df['SecurityId'] =  df.apply(lambda p: getIdnType(p)[0], axis=1)
df['SecurityIdType'] =  df.apply(lambda p: getIdnType(p)[1], axis=1)
df[['SecurityId', 'SecurityIdType', 'PeriodEndDate', 'LongShortDescription', 'SortKey', 'Quantity']].head()

,SecurityId,SecurityIdType,PeriodEndDate,LongShortDescription,SortKey,Quantity
0,CNY,Cash and Equivalents,2020-05-29,Cash Long,Cash and Equivalents,100.00
1,EUR,Cash and Equivalents,2020-05-29,Cash Long,Cash and Equivalents,200713.05
2,HKD,Cash and Equivalents,2020-05-29,Cash Long,Cash and Equivalents,12670525.93
3,USD,Cash and Equivalents,2020-05-29,Cash Long,Cash and Equivalents,42817706.21
4,1299 HK Equity,TICKER,2020-05-29,Investments Long,Common Stock,177200.00


## Prepare Blp Data

1. Write csv of security Ids that needs to load Bloomberg data;
2. Load Bloomberg data into dataframe;
3. Combine with portfolio.

### Write Security Id and Type to Csv File

1. Take those need Bloomberg data;
2. Write the unique security id and type to csv file.

In [23]:
from risk_report.main_asset_allocation import needBlpData

df[df.apply(needBlpData, axis=1)].groupby('SecurityId').first()['SecurityIdType'].to_csv('idnType_' + date + '.csv')

### Get Bloomberg Data and Merge with Positions

In [24]:
from risk_report.data import getBlpData

blpData = dictListToDataFrame(getBlpData(date, mode).values())

df = df.merge(blpData, left_on='SecurityId', right_on='ID', how='left')
df.head()

,index_x,BookCurrency,PeriodEndDate,PeriodStartDate,Portfolio,ReportMode,LongShortDescription,SortKey,LocalCurrency,BasketInvestDescription,...,MARKET_SECTOR_DES,CNTRY_ISSUE_ISO,CNTRY_OF_RISK,EXCH_MARKET_STATUS,INDUSTRY_SECTOR,CAPITAL_CONTINGENT_SECURITY,SFC_AUTHORIZED_FUND,RTG_SP,RTG_MOODY,RTG_FITCH
0,0,HKD,2020-05-29,1950-01-01,19437,Investments,Cash Long,Cash and Equivalents,Chinese Renminbi Yuan,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,HKD,2020-05-29,1950-01-01,19437,Investments,Cash Long,Cash and Equivalents,Euro,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,HKD,2020-05-29,1950-01-01,19437,Investments,Cash Long,Cash and Equivalents,Hong Kong Dollar,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,HKD,2020-05-29,1950-01-01,19437,Investments,Cash Long,Cash and Equivalents,United States Dollar,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,HKD,2020-05-29,1950-01-01,19437,Investments,Investments Long,Common Stock,Hong Kong Dollar,,...,Equity,HK,HK,ACTV,Financial,N,#N/A Field Not Applicable,#N/A,#N/A,#N/A
